In [1]:
import tensorflow as tf
from einops.layers.tensorflow import Rearrange


class MultiHeadedAttention(tf.keras.Model):

    def __init__(self, dimension, heads=8):
        super(MultiHeadedAttention, self).__init__()

        self.heads = heads
        self.scale = dimension ** -0.5

        self.mlp_in = tf.keras.layers.Dense(dimension * 3, use_bias=False)
        self.mlp_out = tf.keras.layers.Dense(dimension)

        self.rearrange_attention = Rearrange(
            'b n (qkv h d) -> qkv b h n d', qkv=3, h=self.heads)
        self.rearrange_output = Rearrange('b h n d -> b n (h d)')

    def call(self, inputs):
        query_key_value = self.mlp_in(inputs)
        query_key_value = self.rearrange_attention(query_key_value)

        query = query_key_value[0]
        key = query_key_value[1]
        value = query_key_value[2]

        dot_product = tf.einsum('bhid,bhjd->bhij', query, key) * self.scale
        attention = tf.nn.softmax(dot_product, axis=-1)

        output = tf.einsum('bhij,bhjd->bhid', attention, value)
        output = self.rearrange_output(output)
        return self.mlp_out(output)

In [5]:
import tensorflow as tf
from echoAI.Activation.TF_Keras.custom_activation import GELU


class ResidualBlock(tf.keras.Model):

    def __init__(self, residual_function):
        super(ResidualBlock, self).__init__()
        self.residual_function = residual_function

    def call(self, inputs):
        return self.residual_function(inputs) + inputs


class NormalizationBlock(tf.keras.Model):

    def __init__(self, norm_function, epsilon=1e-5):
        super(NormalizationBlock, self).__init__()
        self.norm_function = norm_function
        self.normalize = tf.keras.layers.LayerNormalization(epsilon=epsilon)

    def call(self, inputs):
        return self.norm_function(self.normalize(inputs))


class MLPBlock(tf.keras.Model):

    def __init__(self, output_dimension, hidden_dimension):
        super(MLPBlock, self).__init__()
        self.mlp_1 = tf.keras.layers.Dense(hidden_dimension)
        self.mlp_2 = tf.keras.layers.Dense(output_dimension)
        self.activation = GELU()

    def call(self, inputs):
        y = self.mlp_1(inputs)
        y = self.activation(y)
        y = self.mlp_2(y)
        return self.activation(y)

ImportError: cannot import name 'Relu' from 'echoAI.Activation.TF_Keras.custom_activation' (C:\ProgramData\Anaconda3\lib\site-packages\echoAI\Activation\TF_Keras\custom_activation.py)